# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint
import scipy.stats as st

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
loadPath = '../WeatherPy/weather_data.csv'

weatherFrame = pd.read_csv(loadPath)

print(weatherFrame.head())

           City    Lat     Lon  Max Temp  Humidity  Cloudiness  Wind Speed  \
0         Pisco -13.70  -76.22     63.00        82           0       13.87   
1        Lompoc  34.64 -120.46     64.99        52           1        6.85   
2        Faanui -16.48 -151.75     78.46        80           4       11.86   
3  Norman Wells  65.28 -126.83     23.00        67          75        3.36   
4  Ponta do Sol  32.67  -17.10     69.80        73          20       16.11   

  County        Date  
0     PE  1602560253  
1     US  1602560096  
2     PF  1602560254  
3     CA  1602560254  
4     PT  1602560254  


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

humidityFrame = weatherFrame[['Lat', 'Lon', 'Humidity']]

locations = humidityFrame[['Lat', 'Lon']]
weights = humidityFrame['Humidity']

print(humidityFrame.head())

     Lat     Lon  Humidity
0 -13.70  -76.22        82
1  34.64 -120.46        52
2 -16.48 -151.75        80
3  65.28 -126.83        67
4  32.67  -17.10        73


In [4]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowedWeatherData = weatherFrame.loc[(weatherFrame['Humidity'] < 40) & (weatherFrame['Cloudiness'] < 20) & (weatherFrame['Max Temp'] < 85) & (weatherFrame['Max Temp'] > 70) & (weatherFrame['Wind Speed'] < 10)]

print(len(narrowedWeatherData))

10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowedWeatherData

hotel_df['Hotel Name']

print(hotel_df.head())

KeyError: 'Hotel Name'

In [7]:
gApiString = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
pp = pprint.PrettyPrinter(depth=6)
finalTable = hotel_df

for index, row in hotel_df.iterrows():
    gApiString = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={Lat},{Lon}&radius=5000&type=lodging&key={api}'
    
    hotel_response = requests.get(gApiString.format(Lat=row.Lat, Lon=row.Lon, api=g_key))
    hotel_json = hotel_response.json()
    
    if len(hotel_json['results']) > 0:
        hotelName = hotel_json['results'][0]['name']
        print(hotelName)
        finalTable.at[index, 'Hotel Name'] = hotelName
    else:
        #finalTable.at[index, 'Hotel Name'] = 'No Accomodations Available'
        print(index)
        finalTable = finalTable.drop(index)
    
pp.pprint(finalTable)

58
Best Western Plus Abbey Inn
Shellharbour Resort & Conference Centre
Valle Aridane
Chez Honorable Chapiou
Hotel Rural El Patio
387
Eaka Hotel Hengshui Railway Station Branch
Expeditions Ténére voyages
414
                      City    Lat     Lon  Max Temp  Humidity  Cloudiness  \
107           Saint George  37.10 -113.58     73.40        17           1   
196               Flinders -34.58  150.86     84.99        39           3   
211  Los Llanos de Aridane  28.66  -17.92     77.00        38           6   
239                Magaria  13.00    8.91     82.49        20           0   
315      Icod de los Vinos  28.37  -16.71     75.20        23           3   
398               Hengshui  37.73  115.70     73.22        20           0   
404                  Bilma  18.69   12.92     81.25        20           4   

     Wind Speed County        Date                                  Hotel Name  
107        5.82     US  1602560248                 Best Western Plus Abbey Inn  
196        4.0

In [8]:
finalTable = finalTable.rename(columns={'County' : 'Country'})

pp.pprint(finalTable)

                      City    Lat     Lon  Max Temp  Humidity  Cloudiness  \
107           Saint George  37.10 -113.58     73.40        17           1   
196               Flinders -34.58  150.86     84.99        39           3   
211  Los Llanos de Aridane  28.66  -17.92     77.00        38           6   
239                Magaria  13.00    8.91     82.49        20           0   
315      Icod de los Vinos  28.37  -16.71     75.20        23           3   
398               Hengshui  37.73  115.70     73.22        20           0   
404                  Bilma  18.69   12.92     81.25        20           4   

     Wind Speed Country        Date  \
107        5.82      US  1602560248   
196        4.00      AU  1602560302   
211        8.05      ES  1602560305   
239        7.87      NE  1602560311   
315        9.17      ES  1602560327   
398        8.25      CN  1602560350   
404        5.44      NE  1602560351   

                                     Hotel Name  
107                 

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in finalTable.iterrows()]
locations = finalTable[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))